__[Open and try this file online (Colab)](https://colab.research.google.com/github/djekra/wiktionary_wortschatz/blob/master/1_wiktionary_parsen/x20_d00_mwparserfromhell.ipynb)__

# mwparserfromhell
* erstellt die Dateien wiktionary_merkmal.pickle und wiktionary_merkmal_text.pickle in data_00
* parst dafür wiktionary.wikitext mit mwparserfromhell
* File wiktionary wird nicht neu gespeichert

## Import

In [1]:
# blab init
try:
    import blab
except ImportError as e:
    !pip install blab
    import blab    
startup_notebook = blab.blab_startup()
%run $startup_notebook 

blab init
environment['in_colab']     = False
environment['dropbox_path'] = /home/me/Data_Linux/Dropbox
environment['lib_path']     = /home/me/Data_Linux/Dropbox/31_Projekte/01_Python/libs
Start Time: 13:24:16


time: 624 ms


In [2]:
# alle anderen
try:
    import pandasklar as pak, bj_nlp, mwparserfromhell, swifter

except ImportError as e:
    !pip install pandasklar bj_nlp mwparserfromhell swifter
    import pandasklar as pak, bj_nlp, mwparserfromhell, swifter
    
grid       = pak.grid
check_mask = pak.check_mask      

time: 4.2 s


## Einstellungen

In [3]:

version = '_all'
#version = '_short'
speicher_freigeben = True    # sollen alte Ergebnisse gelöscht werden

# Load
verzeichnis_load                 = 'data_00'
wiktionary_filename              = verzeichnis_load + '/wiktionary'               + version + '.pickle'  
wiktionary_trash_filename        = verzeichnis_load + '/wiktionary_trash'         + version + '.pickle'

# Save
verzeichnis_save                 = 'data_00'
wiktionary_merkmal_filename      = verzeichnis_save + '/wiktionary_merkmal'      + version + '.pickle'  
wiktionary_merkmal_text_filename = verzeichnis_save + '/wiktionary_merkmal_text' + version + '.pickle'  

# Was debuggen?
suche_debug = ['entlassen','missraten']

time: 51.6 ms


## Vorbereiten

In [4]:
# Verzeichnisse erzeugen
os.makedirs(verzeichnis_load, exist_ok=True) 
os.makedirs(verzeichnis_save, exist_ok=True) 

time: 44.4 ms


In [5]:
# Steuertabellen und Python-Files von Github herunterladen
import os
if not os.path.exists('Steuertabellen'):
   !git clone https://github.com/djekra/wiktionary_wortschatz.git
   !cp -r wiktionary_wortschatz/1_wiktionary_parsen/Steuertabellen Steuertabellen
   !cp -r wiktionary_wortschatz/1_wiktionary_parsen/*.py .
#   !rm -r wiktionary_wortschatz

time: 38.9 ms


In [6]:
# Lade Ergebnisse der vorherigen Verarbeitungsstufe von Google Drive, falls nicht lokal vorhanden
# Dieses Verzeichnis wurde in der vorherigen Verarbeitungsstufe angelegt (Notebook x10_d00_wiktionary_de_parser)
if environment['in_colab']:
    from google.colab import drive
    drive.mount('/content/drive') 
    !cd /content
    !cp -av /content/drive/MyDrive/{verzeichnis_load} .

time: 43.8 ms


In [7]:
# Weitere Imports 
from Steuertabellen.s01_Steuertabellen   import *

# mwparserfromhell eigene Ergänzungen 
from x24_process_wikicode import *
from x22_my_node          import *

time: 51 ms


from pandarallel import pandarallel
nb_workers = os.cpu_count()
if nb_workers > 2:
    nb_workers = int((nb_workers / 2) - 2)
pandarallel.initialize(use_memory_fs=True, nb_workers=nb_workers, progress_bar=True)  

In [8]:
# Visualisiert einen Wikicode
def see(wikicode):
    return wikicode._get_tree(wikicode,list(),True,0)    

time: 37.4 ms


## Laden

In [9]:
# Laden
wiktionary       = pak.load_pickle(wiktionary_filename)
wiktionary_trash = pak.load_pickle(wiktionary_trash_filename)

time: 9.43 s


In [10]:
print('Einträge:', wiktionary.shape[0])

Einträge: 837593
time: 50.9 ms


In [11]:
# wiktionary anschauen
mask = wiktionary.title.isin(suche_debug) 
r = wiktionary[mask].sort_values(['title','section_id2'])
grid(r)

2 rows


time: 372 ms


In [12]:
pak.analyse_cols(wiktionary)

,col_name,datatype_instance,datatype,datatype_short,datatype_suggest,is_numeric,is_string,is_hashable,nan_allowed,mem_usage,ntypes,nunique,nnan,ndups,n,vmin,vmean,vmedian,vmax,vsum
0,__index__,int64,np.int64,int64,np.int32,True,False,True,False,80.0 B,1,837593,0,0,837593,0.0,418796.00,418796.0,837592.0,3.507806e+11
1,section_id,int32,pd.Int32,Int32,,True,False,True,True,50.0 B,1,837593,0,0,837593,0.0,62209823.28,62312700.0,111831800.0,5.210651e+13
2,page_id,int32,pd.Int32,Int32,,True,False,True,True,50.0 B,1,802086,0,35507,837593,0.0,622098.23,623127.0,1118318.0,5.210651e+11
3,page_part,int8,pd.Int8,Int8,,True,False,True,True,20.0 B,1,7,0,837586,837593,0.0,0.04,0.0,6.0,3.748300e+04
4,section_id2,str,pd.string,string,,False,True,True,True,763.0 B,1,837593,0,0,837593,&c.,NaN,NaN,μm,NaN
5,title,str,pd.string,string,,False,True,True,True,761.0 B,1,802084,0,35509,837593,&c.,NaN,NaN,μm,NaN
6,lemma,str,pd.string,string,,False,True,True,True,749.0 B,1,159601,0,677992,837593,&c.,NaN,NaN,μm,NaN
7,inflected,bool_,bool,bool,,True,False,True,True,10.0 B,1,2,0,837591,837593,0.0,0.83,NaN,1.0,6.969110e+05
8,rhymes,list,object,object,,False,True,False,True,512.0 B,1,39529,527175,270889,837593,[a],NaN,NaN,"[ʏʃəs, yːʃəs]",NaN
9,pos,dict,object,object,,False,True,False,True,2.2 KB,1,163,0,837430,837593,NaN,NaN,NaN,NaN,NaN


time: 15.7 s


In [13]:
pak.memory_consumption(locals())

,name,rtype,size
0,wiktionary,"(DataFrame, Series)",3.0 GB
1,wiktionary_trash,"(DataFrame, Series)",16.1 MB
2,mask,"(Series, bool)",818.1 KB
3,translate_tag,"(DataFrame, Series)",37.1 KB
4,translate_tagZ,"(DataFrame, Series)",20.9 KB
5,r,"(DataFrame, Series)",20.7 KB
6,_,"(DataFrame, Series)",16.3 KB
7,_12,"(DataFrame, Series)",16.3 KB
8,_i3,"(str,)",1.4 KB
9,Stop,"(type,)",1.0 KB


time: 1.3 s


## Steuertabelle plan_merkmal

In [14]:
blab.help(plan_merkmal_erstellen)

<class 'IPython.core.display.Markdown'>


<span style="font-size:larger;">plan_merkmal_erstellen():</span>

* definiert in Steuertabellen/s01_Steuertabellen.py
* `merkmal`: Name des Merkmals
* `template`: Name des Mediawiki-Templates, das diese Information enthält
* `collect`: Extractionsmethode
   * L extrahiert nur Links 
   * T extrahiert komplette Text-Absätze
   * Ü extrahiert Übersetzungen
   * P extrahiert Template-Parameter (Substring reicht)
   * E checkt nur auf Existenz
* `plan`:
   * 1: die Merkmale, die in wiktionary_merkmal gespeichert werden
   * 2: die Merkmale, die in wiktionary_merkmal_text gespeichert werden   
   * 3: Reihenfolge und Liste aller Merkmale aus flexion / übersicht <br>
        sowie die Merkmale aus named_entities und lexeme_manuell
* `sort`: Irgendwas zum Sortieren der Merkmale
* `is_lex`: Ist das Merkmal ein Lexem?    

time: 62.3 ms


In [15]:
plan_merkmal = plan_merkmal_erstellen()
grid(plan_merkmal)

103 rows


time: 176 ms


## ausprobieren

In [16]:
# Eine Testzeile aus wiktionary (als Series)
mask = (wiktionary.section_id2 == 'Haus')
testzeile = wiktionary[mask].iloc[0]
wikicode = mwparserfromhell.parse(testzeile.wikitext)

# testzeile 
# testzeile.wikitext
# wikicode

time: 381 ms


In [17]:
# extract_parameters ausprobieren (funktioniert z.B. mit 'kein')
r = extract_parameters(testzeile, 'Pronomina-Tabelle', 'übersicht', wikicode=wikicode )
grid(r, color=None)

No rows
time: 41.4 ms


In [18]:
#r.node_debug = r.node_debug.astype('str')
pak.analyse_cols(r)

,col_name,datatype_instance,datatype,datatype_short,datatype_suggest,is_numeric,is_string,is_hashable,nan_allowed,mem_usage,ntypes,nunique,nnan,ndups,n,vmin,vmean,vmedian,vmax,vsum
0,__index__,,object,object,,False,True,True,True,0.0 B,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN


time: 64.5 ms


In [19]:
# extract ausprobieren
df = extract(testzeile, 'Unterbegriffe', 'down', wikicode=wikicode)
grid(df)

412 rows


time: 133 ms


In [20]:
# extract_all ausprobieren
mask = plan_merkmal.plan == 2
plan = plan_merkmal[mask]
df = extract_all(testzeile, plan) 
df

,node_num,node_meta,node_link,node_text,node_kontext,node_debug,section_id,section_id2,merkmal
0,[1],,,zu einem bestimmten Zweck erbautes Gebäude,,,21700,Haus,def
1,[2],,,zum Wohnen dienendes und genutztes Gebäude,,,21700,Haus,def
2,[3],,,"aus mehreren Räumen bestehender, abgetrennter ...",,,21700,Haus,def
3,[4],umgangssprachlich,,umgangssprachlich Gesamtheit der Bewohner in ...,,,21700,Haus,def
4,[5],,,"Gesamtheit der Personen, die sich in einem bes...",,,21700,Haus,def
...,...,...,...,...,...,...,...,...,...
5,"[2, 3]",,,mein Haus ist meine Burg,,,21700,Haus,bsp_sp
6,"[2, 3, 7, 9]",,,ein Haus kann nicht zwei Hunde nähren,,,21700,Haus,bsp_sp
7,[3],,,"lässt du einen ins Haus kommen, er kommt dir b...",,,21700,Haus,bsp_sp
8,[3],,,"Nord, Ost, Süd, West, zu Haus ist’s am best",,,21700,Haus,bsp_sp


time: 1.05 s


## wiktionary_merkmal_text

In [21]:
# plan
mask = plan_merkmal.plan == 2
plan = plan_merkmal[mask]
plan

,merkmal,template,collect,plan,sort,is_lex
38,def,Bedeutungen,T,2,,False
39,herkunft,Herkunft,T,2,,False
40,bsp,Beispiele,T,2,,False
41,bsp_re,Redewendungen,T,2,,False
42,bsp_gw,Geflügelte Worte,T,2,,False
43,bsp_sp,Sprichwörter,T,2,,False


time: 41.8 ms


In [22]:
# all: 40 min (4 parallel) 
# 100K: 3 min
# apply erzeugt eine Series von DataFrames
# tolist macht daraus eine Liste
# und concat macht aus der Liste von DataFrames ein Gesamt-DataFrame
#wiktionary_merkmal_text = pd.concat(  wiktionary.parallel_apply(extract_all, plan=plan, axis=1).tolist()  )
wiktionary_merkmal_text = pd.concat(  wiktionary.swifter.apply(extract_all, plan=plan, axis=1).tolist()  )

Pandas Apply:   0%|          | 0/837593 [00:00<?, ?it/s]

Executing shutdown due to inactivity...


2023-02-16 14:24:52,059 - INFO     - Executing shutdown due to inactivity...
IOStream.flush timed out
IOStream.flush timed out


Executing shutdown...


2023-02-16 14:25:12,759 - INFO     - Executing shutdown...


Exception on /shutdown [GET]
Traceback (most recent call last):
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/app.py", line 435, in shutdown
    shutdown_server()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/app.py", line 421, in shutdown_server
    raise RuntimeError("Not 

2023-02-16 14:25:12,762 - ERROR    - Exception on /shutdown [GET]
Traceback (most recent call last):
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/app.py", line 435, in shutdown
    shutdown_server()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/app.py", line 421, in shutdow

KeyboardInterrupt: 

time: 2h 53min 34s


In [ ]:
# nachbearbeiten
wiktionary_merkmal_text = pak.reset_index( wiktionary_merkmal_text )
wiktionary_merkmal_text = pak.move_cols(   wiktionary_merkmal_text, ['section_id','section_id2','merkmal'])
wiktionary_merkmal_text = pak.drop_cols(   wiktionary_merkmal_text, ['node_link','node_kontext','node_debug'])
wiktionary_merkmal_text = pak.rename_col(  wiktionary_merkmal_text, 'node_num',  'num')
wiktionary_merkmal_text = pak.rename_col(  wiktionary_merkmal_text, 'node_meta', 'meta')
wiktionary_merkmal_text = pak.rename_col(  wiktionary_merkmal_text, 'node_text', 'data')    

## Kontrolle: wiktionary_merkmal_text

In [ ]:
# Merkmale Häufigkeit
v = pak.analyse_freqs(wiktionary_merkmal_text, 'merkmal') 
v

In [ ]:
# plan erfüllt?
geplant = set(plan.merkmal)
done    = set(v.merkmal)
nicht_erfüllt = geplant - done
print(nicht_erfüllt)
assert len(nicht_erfüllt) == 0

In [ ]:
# Sichtkontrolle
mask = wiktionary.section_id2.isin(suche_debug)
suche_section_ids = wiktionary[mask].section_id.tolist()
mask = wiktionary_merkmal_text.section_id.isin(suche_section_ids)
wiktionary_merkmal_text[mask]

## Speichern

In [ ]:
# Speichern   
if wiktionary_merkmal_text_filename:
    pak.dump_pickle(wiktionary_merkmal_text, wiktionary_merkmal_text_filename)   

In [ ]:
# Speicher
if speicher_freigeben:
    try:
        del wiktionary_merkmal_text
    except:
        pass
pak.memory_consumption(locals())

In [ ]:
#pak.analyse_cols(wiktionary)

## wiktionary_merkmal

In [ ]:
# plan
mask = plan_merkmal.plan == 1
plan = plan_merkmal[mask]
plan

In [ ]:
# all: 2h 22min
# 100K: 8min 44s
# ERROR1 2ERROR
# apply erzeugt eine Series von DataFrames
# tolist macht daraus eine Liste
# und concat macht aus der Liste von DataFrames ein Gesamt-DataFrame
#wiktionary_merkmal = pd.concat(  wiktionary.parallel_apply(extract_all, plan=plan, axis=1).tolist()  )
wiktionary_merkmal = pd.concat(  wiktionary.swifter.apply(extract_all, plan=plan, axis=1).tolist()  )

In [ ]:
# nachbearbeiten
wiktionary_merkmal = pak.reset_index( wiktionary_merkmal )
wiktionary_merkmal = pak.move_cols(   wiktionary_merkmal, ['section_id','section_id2','merkmal'])
wiktionary_merkmal = pak.drop_cols(   wiktionary_merkmal, ['node_link'])
wiktionary_merkmal = pak.rename_col(  wiktionary_merkmal, 'node_num',  'num')
wiktionary_merkmal = pak.rename_col(  wiktionary_merkmal, 'node_meta', 'meta')
wiktionary_merkmal = pak.rename_col(  wiktionary_merkmal, 'node_text', 'data')    

## Kontrolle: wiktionary_merkmal

In [ ]:
# Merkmale Häufigkeit
v = pak.analyse_freqs(wiktionary_merkmal, 'merkmal') 
v

In [ ]:
# plan erfüllt?
geplant = set(plan.merkmal)
done    = set(v.merkmal)
nicht_erfüllt = geplant - done
print(nicht_erfüllt)
if version == '_all': 
    assert len(nicht_erfüllt) <= 1  # {'reim'} ist nicht erfüllt

In [ ]:
# Suche festlegen
suche_debug = ['Haus',]

In [ ]:
# Alle Merkmale einer Section
mask = wiktionary.section_id2.isin(suche_debug)
suche_section_ids = wiktionary[mask].section_id.tolist()
mask = wiktionary_merkmal.section_id.isin(suche_section_ids)
a = pak.analyse_freqs( wiktionary_merkmal[mask], ['merkmal','data'], limits=[999,None])
#grid(a)
a

In [ ]:
# Alle Merkmale einer Section
mask = wiktionary.title.isin(suche_debug)
suche_section_ids = wiktionary[mask].section_id.tolist()
mask = wiktionary_merkmal.section_id.isin(suche_section_ids)
#grid(wiktionary_merkmal, mask)
wiktionary_merkmal[mask]

In [ ]:
# Ein Merkmal in allen Sections
mask = (wiktionary_merkmal.merkmal == 'up_part')
#grid(wiktionary_merkmal, mask)
wiktionary_merkmal[mask]

## Müll löschen

In [ ]:
# Reste von Bildern
mask1 = (wiktionary_merkmal.merkmal == 'übersicht')
mask2 = (wiktionary_merkmal.node_kontext.str.len() < 3)
mask = mask1  &  mask2
wiktionary_merkmal = pak.drop_rows(wiktionary_merkmal,mask, verbose=True)

In [ ]:
# wiktionary_merkmal: Spalten löschen
wiktionary_merkmal = pak.drop_cols(wiktionary_merkmal,['section_id'])

## Speichern

In [ ]:
# Speichern
if wiktionary_merkmal_filename:
    pak.dump_pickle(wiktionary_merkmal, wiktionary_merkmal_filename) 

In [ ]:
# Ausgabeverzeichnis touch
import os
os.utime(verzeichnis_save)

In [ ]:
# Save in Google Drive
if environment['in_colab']:
    from google.colab import drive
    drive.mount('/content/drive') 
    !cd /content
    !cp -av {verzeichnis_save} /content/drive/MyDrive

In [ ]:
# memory_consumption
pak.memory_consumption( locals() )